In [4]:
import pandas as pd

df = pd.read_parquet('..\Project Main\data\Appliance_file_subset.parquet')

df.drop(['images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase'], axis=1, inplace=True)

df.head()



FileNotFoundError: [Errno 2] No such file or directory: '..\\Project Main\\data\\Appliance_file_subset.parquet'

In [1]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

# Example preprocessing
def encode_review(title, description):
    review_text = title + " " + description
    return tokenizer(review_text, padding=True, truncation=True, return_tensors="tf")

# Example input data
encoded_review = encode_review("Great product", "This product exceeded my expectations.")
output = model(encoded_review['input_ids'])

# Compile the model
model.compile(optimizer=Adam(learning_rate=5e-5), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Fit the model on your training data
model.fit(train_dataset, epochs=3)


C:\Users\lonni\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
No module named 'keras.__internal__'

In [4]:
! pip install keras





: 

In [2]:
import numpy as np
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import pandas as pd


# Load your data
df = pd.read_parquet('..\Project Main\data\Appliance_file_subset.parquet')
df.drop(['images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase'], axis=1, inplace=True)

# Combine the title and text into one feature
df['combined_text'] = df['title'] + ' ' + df['text']

# Prepare tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)

# Tokenize the text data
def tokenize_function(texts):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="tf"
    )

# Prepare the inputs
X = df['combined_text'].tolist()
y = df['rating'].astype(int) - 1  # Ratings should be 0-based for the model (e.g., 0 for 1-star, 4 for 5-star)

# Tokenize the text inputs
X_tokenized = tokenize_function(X)

# Convert tokenized input tensors to numpy arrays
input_ids = X_tokenized['input_ids'].numpy()

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(input_ids, y, test_size=0.2, random_state=42)

# Compile the model
model.compile(optimizer=Adam(learning_rate=5e-5), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Train the model
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=3,
    batch_size=16
)

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation accuracy: {accuracy}')


C:\Users\lonni\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.models.roberta.modeling_tf_roberta because of the following error (look up to see its traceback):
No module named 'keras.__internal__'

In [1]:
import numpy as np
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from sklearn.model_selection import train_test_split
import tensorflow as tf
# from tensorflow.keras.optimizers import Adam
import pandas as pd

2024-10-25 16:51:20.002566: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-25 16:51:20.009742: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-25 16:51:20.018327: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-25 16:51:20.020841: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-25 16:51:20.027506: I tensorflow/core/platform/cpu_feature_guar

In [2]:
df

,rating,title,text
0,5.0,Excellent.,Quality Product.
1,5.0,Five Stars,Works as expected.
2,5.0,Perfectly fit my Vornado,These filters are a perfect fit for my Vornado...
3,5.0,Coffee Filter,This filter was just as good as the original o...
4,5.0,Liked everything about k cups,Liked everything about k cups. These are much ...
...,...,...,...
49995,5.0,Fits and works perfectly on my refrigerator,It fits and works perfectly on my fridge with ...
49996,5.0,BEST FILTER OUT THERE!!!!!!!!,I had on of the gold mesh/plastic framed filte...
49997,5.0,Will never buy the aluminum ones again,Perfect to replace cheap aluminum ones on our ...
49998,5.0,great,nice


In [40]:
df

,rating,title,text,combined_text
0,5.0,Excellent.,Quality Product.,excellent quality product
1,5.0,Five Stars,Works as expected.,five starsworks expected
2,5.0,Perfectly fit my Vornado,These filters are a perfect fit for my Vornado...,perfectly fit vornadothese filters perfect fit...
3,5.0,Coffee Filter,This filter was just as good as the original o...,coffee filterthis filter good original one cam...
4,5.0,Liked everything about k cups,Liked everything about k cups. These are much ...,liked everything k cupsliked everything k cups...
...,...,...,...,...
49995,5.0,Fits and works perfectly on my refrigerator,It fits and works perfectly on my fridge with ...,fits works perfectly refrigeratorit fits works...
49996,5.0,BEST FILTER OUT THERE!!!!!!!!,I had on of the gold mesh/plastic framed filte...,best filter gold mesh plastic framed filters y...
49997,5.0,Will never buy the aluminum ones again,Perfect to replace cheap aluminum ones on our ...,never buy aluminum ones againperfect replace c...
49998,5.0,great,nice,greatnice


In [39]:
import tensorflow as tf
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from sklearn.model_selection import train_test_split
import pandas as pd

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

# define characters to remove and stop words
regex = re.compile("[^a-zA-Z ]")
stop_words = set(stopwords.words('english'))

# function to clean text
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = regex.sub(' ', text) # Substitute everything that is not a letter with an empty string
    words = word_tokenize(text) # tokenize text
    words = [word for word in words if word not in stop_words]  # Remove stopwords
    return ' '.join(words)


# GPU check
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Load your data
df = pd.read_parquet('../Project Main/data/Appliance_file_subset.parquet')
df.drop(['images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase'], axis=1, inplace=True)
#combined_text = df['title'] + df['text']
df['combined_text'] = [preprocess_text(title + text) for title, text in zip(df['title'], df['text'])]



[nltk_data] Downloading package stopwords to /home/mike/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mike/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Num GPUs Available:  1


In [43]:

# Prepare tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)

# Tokenize the text data
def tokenize_function(texts):
    return tokenizer(
        texts,
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors="tf"
    )

# Prepare the inputs and labels
X = df['combined_text'].tolist()
y = tf.convert_to_tensor(df['rating'].astype(int).values - 1, dtype=tf.int32)  # Ensure labels are TensorFlow-compatible
X_tokenized = tokenize_function(X)

# Convert input_ids and attention_mask to NumPy arrays
input_ids = X_tokenized['input_ids'].numpy()
attention_mask = X_tokenized['attention_mask'].numpy()
y = y.numpy()

# Split the dataset into training and validation sets
X_train_ids, X_val_ids, X_train_mask, X_val_mask, y_train, y_val = train_test_split(
    input_ids, attention_mask, y, test_size=0.2, random_state=42
)

# Convert the data to a TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': X_train_ids, 'attention_mask': X_train_mask}, y_train)).batch(16)
val_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': X_val_ids, 'attention_mask': X_val_mask}, y_val)).batch(16)

# Compile the model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Train the model (inside GPU context)
with tf.device('/GPU:0'):
    model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=3
    )

# Evaluate the model
loss, accuracy = model.evaluate(val_dataset)
print(f'Validation accuracy: {accuracy}')


/home/mike/anaconda3/envs/dev/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassific

Epoch 1/3


I0000 00:00:1730135352.506920   30210 service.cc:146] XLA service 0x7f0864511780 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730135352.507005   30210 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-10-28 13:09:12.519455: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-28 13:09:12.544582: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902
I0000 00:00:1730135352.604397   30210 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2500/2500 [==============================] - 551s 212ms/step - loss: 1.0232 - accuracy: 0.6973 - val_loss: 1.0097 - val_accuracy: 0.6968
Epoch 2/3
2500/2500 [==============================] - 526s 210ms/step - loss: 1.0101 - accuracy: 0.6988 - val_loss: 1.0069 - val_accuracy: 0.6968
Epoch 3/3
625/625 [==============================] - 45s 71ms/step - loss: 1.0083 - accuracy: 0.6968
Validation accuracy: 0.6967999935150146


In [44]:
model.save('../../trained_model')

INFO:tensorflow:Assets written to: ../../trained_model/assets


INFO:tensorflow:Assets written to: ../../trained_model/assets


In [5]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1
